In [ ]:
import cherrypy
import json
import redis
import uuid
import datetime
from redis.commands.json.path import Path


device = 0
host = 'redis-16255.c267.us-east-1-4.ec2.cloud.redislabs.com'
port = 16255
username = 'default'
password = 'J44swf7CVJjEMIjy6tEYuFke5PxxG0kf'

# Connect to Redis server
redis_client = redis.Redis(host=host, port=port, username=username, password=password)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

# endpoint /status
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

# endpoint /batteriesStatus
class BatteryStatus(object):
    exposed = True

    def GET(self, *path, **query):
        keys = redis_client.keys('*')
        monitored = []

        blt = query.get('blt', None)
        plugged = query.get('plugged', None)

        # Validate blt and plugged values
        if blt is not None:
            blt = int(blt)
            if blt < 0 or blt > 100:
                raise cherrypy.HTTPError(400, 'Invalid battery threshold value.')
            
        if plugged is not None:
            plugged = int(plugged)
            if plugged not in [0, 1]:
                raise cherrypy.HTTPError(400, 'Invalid plugged value.')

        devices = {}
        for key in keys:
            key = key.decode()
            mac_address, status_type = key.split(':')

            if mac_address not in devices:
                devices[mac_address] = {}

            # Get all data in the time series and take the last element
            data = redis_client.execute_command('TS.RANGE', key, '-', '+')
            if data:
                devices[mac_address][status_type] = int(data[-1][1])  # Convert to integer

        for mac_address, statuses in devices.items():
            battery_status = statuses.get('battery')
            power_status = statuses.get('power')

            if battery_status is not None and power_status is not None:
                if (blt is None or battery_status <= blt) and \
                        (plugged is None or power_status == bool(plugged)):
                    monitored.append(mac_address)

        response = json.dumps({"mac_addresses": monitored})
        cherrypy.response.status = 200

        return response

# endpoint /batteryStatus/{id}
class batteryStatusId(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        print(mac_address)

        start_date = query.get('start_date')
        end_date = query.get('end_date')
        print(start_date)
        print(end_date)

        if not start_date:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start date')
        if not end_date:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end date')

        try:
            start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        except ValueError:
            raise cherrypy.HTTPError(400, 'Bad Request: wrong format for start date')
        
        try:
            end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
        except ValueError:
            raise cherrypy.HTTPError(400, 'Bad Request: wrong format for end date')

        if end_date <= start_date:
            raise cherrypy.HTTPError(400, 'Bad Request: end date smaller or equal than start date')

        battery_key = f'{mac_address}:battery'
        power_key = f'{mac_address}:power'
        if not (redis_client.exists(battery_key) and redis_client.exists(power_key)):
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')

        start_timestamp = int(start_date.timestamp() * 1000)
        end_timestamp = int(end_date.timestamp() * 1000)

        battery_data = redis_client.execute_command('TS.RANGE', battery_key, start_timestamp, end_timestamp)
        power_data = redis_client.execute_command('TS.RANGE', power_key, start_timestamp, end_timestamp)

        if not battery_data or not power_data:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address or date range')
        item = {
            'mac_address': mac_address,
            'timestamps': [x[0] for x in battery_data],
            'battery_levels': [x[1].decode() if isinstance(x[1], bytes) else x[1] for x in battery_data],
            'power_plugged': [x[1].decode() if isinstance(x[1], bytes) else x[1] for x in power_data],
        }

        response = json.dumps(item)
        cherrypy.response.status = 200

        return response

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        battery_key = f'{mac_address}:battery'
        power_key = f'{mac_address}:power'
        if not (redis_client.exists(battery_key) and redis_client.exists(power_key)):
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')

        redis_client.delete(battery_key)
        redis_client.delete(power_key)

        cherrypy.response.status = 200

        return "Everything worked as expected"

if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Status(), '/online', conf)
    cherrypy.tree.mount(BatteryStatus(), '/devices', conf)
    cherrypy.tree.mount(batteryStatusId(), '/device/', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[17/Jan/2024:11:30:18] ENGINE Bus STARTING
[17/Jan/2024:11:30:18] ENGINE Started monitor thread 'Autoreloader'.
[17/Jan/2024:11:30:18] ENGINE Serving on http://0.0.0.0:8080
[17/Jan/2024:11:30:18] ENGINE Bus STARTED
Redis Connected: True
127.0.0.1 - - [17/Jan/2024:11:30:26] "GET /online HTTP/1.1" 200 20 "" "python-requests/2.28.1"
127.0.0.1 - - [17/Jan/2024:11:30:26] "GET /devices HTTP/1.1" 200 21 "" "python-requests/2.28.1"
127.0.0.1 - - [17/Jan/2024:11:30:27] "GET /devices/?blt=25&plugged=0 HTTP/1.1" 200 21 "" "python-requests/2.28.1"
[17/Jan/2024:11:41:35] ENGINE Keyboard Interrupt: shutting down bus
[17/Jan/2024:11:41:35] ENGINE Bus STOPPING
[17/Jan/2024:11:41:36] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 8080)) shut down
[17/Jan/2024:11:41:36] ENGINE Stopped thread 'Autoreloader'.
[17/Jan/2024:11:41:36] ENGINE Bus STOPPED
[17/Jan/2024:11:41:36] ENGINE Bus EXITING
[17/Jan/2024:11:41:36] ENGINE Bus EXITED
[17/Jan/2024:11:41:36] ENGINE Waiting for child threa

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1b0b0df8-8392-4edb-9f9f-6f8029fb1a2c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>